In [1]:
#%%
from model import Generator, Discriminator, DomainClassifier
from utils import audio_process
import IPython.display as ipd
import tensorflow as tf
import numpy as np
import os

/home/halley/anaconda3/envs/tensorflow_2.1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
#%%
'''
Usage:
  Interactive web page via jupyter
'''
ckpt_path = './ckpt_0'

G = Generator()
dummy_wav = np.zeros((1, 36, 512, 1), dtype=np.float32)
dummy_speaker_onehot = np.zeros((1, 2), dtype=np.float32)
G(dummy_wav, dummy_speaker_onehot)
restored_ckpt = tf.train.Checkpoint(G=G)
latest_ckpt = tf.train.latest_checkpoint(ckpt_path)
restored_ckpt.restore(latest_ckpt).assert_existing_objects_matched()

In [4]:
#%%
result_dir = 'result'
input_wav = '../dataset/speakers/p2/p2_260.wav'
target_speaker = 0
speaker_counts = 2

mfcc = audio_process.wav2mfcc(input_wav)
mfcc = mfcc.astype(np.float32)
mfcc = mfcc[np.newaxis, :, :, np.newaxis]
speaker = np.eye(speaker_counts, dtype=np.float32)[target_speaker]
speaker = speaker[np.newaxis, :]
print(speaker)
print(np.mean(mfcc))
converted = G(mfcc, speaker)
converted_np = converted.numpy()[0,:,:,0]
print(np.mean(converted_np))
converted_wav = audio_process.mfcc2wav(converted_np)

filename = input_wav.split('/')[-1].split('.')[0]
filename = filename + '_to_speaker%d.wav' % (target_speaker)
filepath = os.path.join(ckpt_path, result_dir, filename)
os.makedirs(os.path.dirname(filepath), exist_ok=True)
abs_filepath = audio_process.write_wav(converted_wav, filepath)
ipd.Audio(abs_filepath)

[[1. 0.]]
-4.7514234
0.0563569
